In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder 
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier 
import tensorflow as tf 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import pickle

In [2]:
data = pd.read_csv('Bank_Customer_Churn_Prediction.csv')
data.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
data.drop('customer_id', axis= 1, inplace= True)

label_encoder_gen = LabelEncoder()
data['gender'] = label_encoder_gen.fit_transform(data['gender'])

onehot_encoder_con = OneHotEncoder(handle_unknown= 'ignore')
geo_encoded = onehot_encoder_con.fit_transform(data[['country']]).toarray() 
geo_encoded_df = pd.DataFrame(geo_encoded, columns= onehot_encoder_con.get_feature_names_out(['country']))

data = pd.concat([data.drop('country', axis= 1), geo_encoded_df], axis= 1)

X = data.drop('churn', axis = 1)
y = data['churn']

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size= 0.25, random_state= 42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 


with open('label_encoder_gen_hyp.pkl', 'wb') as file: 
    pickle.dump(label_encoder_gen, file)


with open('onehot_encoder_con_hyp.pkl', 'wb') as file: 
    pickle.dump(onehot_encoder_con, file)

with open('scaler_hyp.pkl', 'wb') as file: 
    pickle.dump(scaler, file)

In [4]:
## define a function to create the model and try different parameters (KerasClassifier)

def create_model(neurons = 32, layers = 1): 
    model = Sequential()
    model.add(Dense(neurons, activation = 'relu', input_shape=(X_train.shape[1], )))
    
    for _ in range(layers -1):
        model.add(Dense(neurons, activation = 'relu'))

    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    return model 

In [8]:
# Create a Keras classifier 
from scikeras.wrappers import KerasClassifier 
model = KerasClassifier(layers = 1, neurons = 32,build_fn= create_model, verbose= 1)

In [9]:
# define the grid search 

param_grid = {
    'neurons': [16, 32, 64, 128], 
    'layers': [1, 2], 
    'epochs': [50, 100]
}

In [10]:
# perform grid search 

grid = GridSearchCV(estimator= model, param_grid= param_grid, n_jobs= -1, cv = 3)
grid_result = grid.fit(X_train, y_train)

# Print the best params 
print("Best: %f, using %s" %(grid_result.best_score_, grid_result.best_params_))

/Users/saivineethgrandhi/Documents/C/GenAI/SimpleANNImplementation/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/saivineethgrandhi/Documents/C/GenAI/SimpleANNImplementation/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/saivineethgrandhi/Documents/C/GenAI/SimpleANNImplementation/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/saivineethgrandhi/Documents/C/GenAI/SimpleANNImplementation/venv/lib/python3.11/site-packages/sc

Epoch 1/50
Epoch 1/50
Epoch 1/50


2025-04-17 08:12:53.320795: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-17 08:12:53.341371: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-17 08:12:53.347223: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50
Epoch 1/50


2025-04-17 08:12:53.854843: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-17 08:12:53.868216: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-17 08:12:53.886425: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-17 08:12:53.896615: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-04-17 08:12:53.937688: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


...

157/157 [==============================] - 4s 24ms/step - loss: 0.5773 - accuracy: 0.7916
Epoch 14/100
157/157 [==============================] - 4s 26ms/step - loss: 0.5813 - accuracy: 0.7818
Epoch 13/100
157/157 [==============================] - 6s 36ms/step - loss: 0.5738 - accuracy: 0.7826
Epoch 20/100
157/157 [==============================] - 7s 42ms/step - loss: 0.4820 - accuracy: 0.7984
Epoch 22/100
157/157 [==============================] - 7s 43ms/step - loss: 0.4949 - accuracy: 0.7996
Epoch 21/100
157/157 [==============================] - 8s 49ms/step - loss: 0.5659 - accuracy: 0.7910
Epoch 15/100
157/157 [==============================] - 7s 47ms/step - loss: 0.6101 - accuracy: 0.7760
Epoch 21/100
157/157 [==============================] - 7s 47ms/step - loss: 0.5346 - accuracy: 0.7852
Epoch 23/100
157/157 [==============================] - 6s 41ms/step - loss: 0.5427 - accuracy: 0.7934
Epoch 16/100
157/157 [==============================] - 7s 41ms/step - loss: 0.7312 - 

/Users/saivineethgrandhi/Documents/C/GenAI/SimpleANNImplementation/venv/lib/python3.11/site-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
2025-04-17 09:00:39.849230: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2025-04-17 09:00:39.858428: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-17 09:00:39.858597: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-17 09:00:39.858809: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-17 09:00:39.858880: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0

Epoch 1/100


2025-04-17 09:00:43.738127: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


235/235 [==============================] - 11s 13ms/step - loss: 0.5256 - accuracy: 0.7559
Epoch 2/100
235/235 [==============================] - 2s 8ms/step - loss: 0.4360 - accuracy: 0.8099
Epoch 3/100
235/235 [==============================] - 2s 9ms/step - loss: 0.4332 - accuracy: 0.8117
Epoch 4/100
235/235 [==============================] - 2s 8ms/step - loss: 0.4338 - accuracy: 0.8105
Epoch 5/100
235/235 [==============================] - 2s 9ms/step - loss: 0.4338 - accuracy: 0.8087
Epoch 6/100
235/235 [==============================] - 2s 7ms/step - loss: 0.4336 - accuracy: 0.8103
Epoch 7/100
235/235 [==============================] - 35s 150ms/step - loss: 0.4334 - accuracy: 0.8113
Epoch 8/100
235/235 [==============================] - 1s 5ms/step - loss: 0.4334 - accuracy: 0.8087
Epoch 9/100
235/235 [==============================] - 1s 5ms/step - loss: 0.4333 - accuracy: 0.8096
Epoch 10/100
235/235 [==============================] - 1s 6ms/step - loss: 0.4333 - accuracy: 0.8